## Day 28 Lecture 2 Assignment

In this assignment, we will learn about overfitting and regularization. We will use the king county housing dataset loaded below and analyze the regression from this dataset.

In [1]:
%load_ext nb_black
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

<IPython.core.display.Javascript object>

In [2]:
king_county = pd.read_csv(
    "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/kc_house_data.csv"
)

<IPython.core.display.Javascript object>

In [3]:
king_county.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


<IPython.core.display.Javascript object>

Perform the same transformations in the previous assignment to meet model assumptions:
1. Remove all columns except: price, bedrooms, bathrooms, sqft_living, floors, waterfront
1. Remove outliers
1. Split the data into train and test subsets. 20% of the data should be in the test subset

In [4]:
# answer below:

king_county2 = king_county[
    ["price", "bedrooms", "bathrooms", "sqft_living", "floors", "waterfront"]
]

<IPython.core.display.Javascript object>

In [5]:
king_county2 = king_county2[king_county2["bedrooms"] != (33)]
king_county2 = king_county2[king_county2["floors"] != (3.5)]

<IPython.core.display.Javascript object>

In [6]:
X = king_county2.drop("price", axis=1)
y = king_county2["price"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<IPython.core.display.Javascript object>

Apply a ridge regression model with lambda=50 to the data and evaluate by looking at r squared for test and train

In [7]:
# answer below:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=50)
ridge.fit(X_train,y_train)




Ridge(alpha=50, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

<IPython.core.display.Javascript object>

In [8]:
print(
    f"R-squared train: {ridge.score(X_train,y_train):.3f}\n"
    f"R-squared test {ridge.score(X_test,y_test):.3f}"
)

R-squared train: 0.532
R-squared test 0.557


<IPython.core.display.Javascript object>

Perform a grid search for the following values of alpha: 0.001, 0.01, 0.1, 1, 10, 100, 1000 to find the most optimal ridge regression model. Experiment with different scoring metrics in the grid search (R^2 is the default, but you can use root mean squared error or many others). 
https://scikit-learn.org/stable/modules/model_evaluation.html

In [9]:
# answer below:
grid = {"alpha": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

from sklearn.model_selection import GridSearchCV

ridge_cv = GridSearchCV(Ridge(), grid, cv=5)
ridge_cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

<IPython.core.display.Javascript object>

In [10]:
print(
    f"alpha: {ridge_cv.best_estimator_.alpha}\n"
    f"R-squared train: {ridge_cv.best_score_:.3f}\n"
    f"R-squared test {ridge_cv.best_score_:.3f}"
)

alpha: 1
R-squared train: 0.528
R-squared test 0.528


<IPython.core.display.Javascript object>

In [11]:
ridge_cv2 = GridSearchCV(Ridge(), grid, cv=5, scoring="neg_root_mean_squared_error")
ridge_cv2.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_root_mean_squared_error', verbose=0)

<IPython.core.display.Javascript object>

In [12]:
print(
    f"alpha: {ridge_cv2.best_estimator_.alpha}\n"
    f"nRMSE train: {ridge_cv2.best_score_:.3f}\n"
    f"nRMSE test {ridge_cv2.best_score_:.3f}"
)

alpha: 1
nRMSE train: -247774.107
nRMSE test -247774.107


<IPython.core.display.Javascript object>

In [13]:
ridge_cv3 = GridSearchCV(Ridge(), grid, cv=5, scoring="neg_mean_absolute_error")
ridge_cv3.fit(X_train, y_train)


GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=0)

<IPython.core.display.Javascript object>

In [15]:
print(
    f"alpha: {ridge_cv3.best_estimator_.alpha}\n"
    f"nMAE train: {ridge_cv3.best_score_:.3f}\n"
    f"nMAE test {ridge_cv3.best_score_:.3f}"
)

alpha: 0.001
nMAE train: -166836.800
nMAE test -166836.800


<IPython.core.display.Javascript object>